In [36]:
import os
import sys
import imp
import config
import numpy as np
from config import *
from time import sleep
from pytz import timezone
from datetime import datetime
from timeout_decorator import timeout, TimeoutError
from poloniex import Poloniex, PoloniexCommandException


def exceptionHandler(timeOut, again):
    def decorator(func):
        @timeout(timeOut)
        def wrapper(*args, **kwargs):
            global agent
            try:
                return func(*args, **kwargs)
            except KeyboardInterrupt:
                print(BLANK, end='\r')
                print(strTime() + ' Good bye.')
                sys.exit()
            except TimeoutError as e:
                print(BLANK, end='\r')
                print(strTime() + ' Time out.')
                agent = Poloniex(apikey, secret)
                sleep(.1)
                if again: return wrapper(*args, **kwargs)
            except (PoloniexCommandException, Exception) as e:
                print(BLANK, end='\r')
                print(strTime(), e)
                agent = Poloniex(apikey, secret)
                sleep(.1)
                if again: return wrapper(*args, **kwargs)
        return wrapper
    return decorator


@exceptionHandler(timeOut=60, again=True)
def sim(path):
    lastState = path[-1]
    amount = lastState[0]
    fromCoin = lastState[1]
    pastCoins = [p[1] for p in path]
    mask = [fromCoin in pair.split('_') for pair in pairs]
    pairs_filtered = pairs[mask]
    pathList = []
    for pair in pairs_filtered:
        temp = pair.split('_')
        if fromCoin == temp[0]: toCoin = temp[1]
        else: toCoin = temp[0]
        pair, rates, amounts, reverse = orderConfig(toCoin, fromCoin, amount)
        non_reverse = 1 - reverse
        amount_ = amounts[non_reverse] * rates[non_reverse]**non_reverse * (1 - feeRate) * (1 - slippage)
        if toCoin == baseCoin: pathList.append(path + [(amount_, toCoin)])
        elif toCoin in pastCoins: continue
        elif len(path) < maxLen - 1:
            temp = sim(path + [(amount_, toCoin)])
            if not temp: continue
            if type(temp[0]) is tuple: pathList.append(temp)
            else: pathList += temp
    if len(pathList) > 1: return pathList
    elif len(pathList) == 1: return pathList[0]


def optimum(pathList):
    retnList = [path[-1][0] / path[0][0] - 1 for path in pathList]
    optRank = np.argmax(retnList)
    optPath = pathList[optRank]
    optRetn = retnList[optRank]
    return optPath, optRetn


@exceptionHandler(timeOut=10, again=False)
def order(baseCoin, coin, amount):
    if (baseCoin == coin) or (amount == 0): return True
    pair, rates, amounts, reverse = orderConfig(baseCoin, coin, abs(amount))
    if ((not reverse) and (amount > 0)) or (reverse and (amount < 0)):  # buy pair, index = 0
        rate = rates[0] * (1 + slippage)
        amount = amounts[0]
        symbol = ' Buy '
        func = agent.buy
    else:  # sell pair, index = 1
        rate = rates[1] * (1 - slippage)
        amount = amounts[1]
        symbol = ' Sell '
        func = agent.sell
    baseAmount = amount * rate**(1 - reverse)
    if baseAmount < minOrder(baseCoin): return True
    while baseAmount >= minOrder(baseCoin):
        print(strTime() + symbol + GRAY + '{:.4f}'.format(amount) + RESET + coin + ' @ ' + GRAY + '{:.4f}'.format(rate) + RESET + baseCoin + '/' + coin)
        amount = func(pair, rate, amount, immediateOrCancel=1)['amountUnfilled']
    return True


@exceptionHandler(timeOut=60, again=True)
def closeAll():
    balances = availableBalances()
    for coin in balances:
        if coin == baseCoin: continue
        order('BTC', coin, -balances[coin])
    if not (baseCoin == 'BTC'):
        balances = availableBalances()
        order(baseCoin, 'BTC', -balances['BTC'])
    return availableBalances()


@exceptionHandler(timeOut=60, again=True)
def calcMv():
    global orderBooks
    for pair in pairs:
        orderBook = agent.returnOrderBook(pair, 1000)
        orderBooks[pair] = {
            'asks': [[float(x[0]), float(x[1])] for x in orderBook['asks']],
            'bids': [[float(x[0]), float(x[1])] for x in orderBook['bids']]
        }
        sleep(.1)
    n_BTC = 0
    balances = closeAll()
    USDT_BTC = orderBooks['USDT_BTC']['asks'][0][0]
    for coin in balances:
        b = balances[coin]
        if coin == 'BTC': n_BTC += b
        elif coin == 'USDT': n_BTC += b / USDT_BTC
        else: n_BTC += b * orderBooks['BTC_' + coin]['asks'][0][0]
    mv = n_BTC * USDT_BTC
    return mv, balances


@exceptionHandler(timeOut=10, again=True)
def availableBalances():
    return agent.returnAvailableAccountBalances(account='exchange')['exchange']


@exceptionHandler(timeOut=10, again=False)
def orderConfig(baseCoin, coin, amount):
    # rates for {pair} to buy/sell {amount} {coin}
    global orderBooks, pairs
    assert amount >= 0, 'Amount cannot be negative.'
    pair = baseCoin + '_' + coin
    if (pair in orderBooks):
        reverse = 0
    else:
        pair = coin + '_' + baseCoin
        reverse = 1
    rates = []
    amounts = []
    types = ['asks', 'bids']
    for t in types:
        cum = 0
        for (p, a) in orderBooks[pair][t]:
            p_or_1 = p**reverse
            cum += a * p_or_1
            if cum >= amount:
                rates.append(p)
                amounts.append(amount / p_or_1)
                break
    if len(rates) < 2:  # depth is not enough
        print(BLANK, end='\r')
        print(strTime() + ' Order book depth of {} not enough, deleted.'.format(pair))
        pairs = np.delete(pairs, np.where(pairs == pair))
        del orderBooks[pair]
        return None
    return pair, rates, amounts, reverse


minOrder = lambda coin: 0.5 if coin == 'USDT' else 0.0001
now = lambda: datetime.now(timezone('Europe/Amsterdam'))
strTime = lambda: '[' + GREEN + now().strftime('%Y-%m-%d %H:%M:%S') + RESET + ']'


@exceptionHandler(timeOut=60, again=False)
def trade():
    global orderBooks, totalRounds, totalSeconds, balances, mv, start, end
    end = now()
    totalSeconds += (end - start).seconds
    totalRounds += 1
    freq = 60 / (totalSeconds / totalRounds)
    start = now()
    pathList = sim([(balances[baseCoin], baseCoin)])
    optPath, optRetn = optimum(pathList)
    lenTrans = len(optPath) - 1
    pathStr = ' \u2192 '.join([BOLD + '{}'.format(state[1]) + RESET for state in optPath])
    print(BLANK, end='\r')
    print(strTime() + ' ' + RED + '{:.4f}'.format(mv) + RESET + 'USD/' + RED + '{:+.2f}'.format(mv / mv0 * 10000 - 10000) + RESET + 'bp [' + '{}'.format(pathStr) + '] ' + CYAN + '{:+.4f}'.  format(optRetn * 10000) + RESET + '\u00b1' + CYAN + '{:.2f}'.format(lenTrans * slippage * 10000) + RESET + 'bp (' + YELLOW + '{:.2f}'.format(freq) + RESET + 'rounds/min)', end='\r')
    if optRetn > threshold:
        print()
        if sys.platform == 'darwin': os.system('afplay /System/Library/Sounds/Ping.aiff')
        for i in range(lenTrans):
            fromCoin = optPath[i][1]
            toCoin = optPath[i + 1][1]
            amount = balances[fromCoin]
            order(toCoin, fromCoin, -amount)
            balances = availableBalances(agent)
        if sys.platform == 'darwin': os.system('afplay /System/Library/Sounds/Tink.aiff')
    sys.stdout.flush()
    mv, balances = calcMv()


RED = '\u001b[1m\u001b[38;5;196m'
GRAY = '\u001b[1m\u001b[38;5;246m'
GREEN = '\u001b[1m\u001b[38;5;46m'
YELLOW = '\u001b[1m\u001b[38;5;220m'
CYAN = '\u001b[1m\u001b[38;5;51m'
BOLD = '\u001b[1m'
RESET = '\u001b[0m'

In [37]:
if __name__ == '__main__':
    print(strTime() + BOLD + ' Coinbot initializing... ' + RESET, end='')
    sys.stdout.flush()
    agent = Poloniex(apikey, secret)
    feeRate = agent.returnFeeInfo()['takerFee']
    pairs = np.array(agent.returnTicker())
    orderBooks = {}
    start = now()
    mv0, balances = calcMv()
    mv = mv0
    totalRounds = 0
    totalSeconds = 0
    print(BOLD + 'succeed' + RESET)
    print(strTime(), '{:<10}:'.format('Base coin'), baseCoin)
    print(strTime(), '{:<10}:'.format('Max length'), maxLen)
    print(strTime(), '{:<10}:'.format('Slippage'), slippage)
    print(strTime(), '{:<10}:'.format('Threshold'), threshold)
    print(strTime(), '{:<10}:'.format('Position'), '{:.4f}USD'.format(mv0))
    while True:
        imp.reload(config)
        from config import *
        BLANK = ' ' * (120 + maxLen * 6)
        trade()

[2017-12-21 01:39:58] Coinbot initializing... succeed
[2017-12-21 01:40:30] Base coin : BTC
[2017-12-21 01:40:30] Max length: 7
[2017-12-21 01:40:30] Slippage  : 0.0001
[2017-12-21 01:40:30] Threshold : 0.01
[2017-12-21 01:40:30] Position  : 44.9175USD
[2017-12-21 01:41:42] Good bye.                                                                                             SDT → BCH → BTC] -16.3091±3.0000bp (1.9355rounds/min)6m1.9355rounds/min))


SystemExit: 

/Users/Allen_Frostline/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2855: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
